In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [5]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [6]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i
    
pairs = []

In [7]:
def get_accuracy_prob(embed_dim):
    
    pairs = []

    for i in vocabs:
        for j in vocabs:
            if i != j:
                pairs.append((i,j))

    indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

    pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
    pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]
    
    sentences_train = []
    sentences_number_train = []
    sentences_test = []
    sentences_number_test = []

    x_masked_train = []
    y_masked_labels_train = []
    x_masked_test = []
    y_masked_labels_test = []

    for _ in range(50000):

        [(a,b), (c,d), (e,f)] = random.sample(pairs_train, 3)

        sentences_train.append([a, b, a, c, d, c, e, f, e])
        sentences_number_train.append([vocab_map[a], vocab_map[b], vocab_map[a], 
                                       vocab_map[c], vocab_map[d], vocab_map[c],
                                       vocab_map[e], vocab_map[f], vocab_map[e]])
        x_masked_train.append([vocab_map[a], vocab_map[b], vocab_map[a], 
                               vocab_map[c], vocab_map[d], vocab_map[c],
                               vocab_map[e], vocab_map[f], vocab_map[e]])
        y_masked_labels_train.append([vocab_map[b], vocab_map[a], 
                                      vocab_map[c], vocab_map[d], vocab_map[c],
                                      vocab_map[e], vocab_map[f], vocab_map[e]])

        [(a,b), (c,d), (e,f)] = random.sample(pairs_test, 3)

        sentences_test.append([a, b, b, c, d, d, e, f, f])
        sentences_number_test.append([vocab_map[a], vocab_map[b], vocab_map[b], 
                                      vocab_map[c], vocab_map[d], vocab_map[d],
                                      vocab_map[e], vocab_map[f], vocab_map[f]])
        x_masked_test.append([vocab_map[a], vocab_map[b], vocab_map[b], 
                              vocab_map[c], vocab_map[d], vocab_map[d],
                              vocab_map[e], vocab_map[f], vocab_map[f]])
        y_masked_labels_test.append([vocab_map[b], vocab_map[b], 
                                     vocab_map[c], vocab_map[d], vocab_map[d],
                                     vocab_map[e], vocab_map[f], vocab_map[f]])

    x_masked_train = np.array(x_masked_train)
    y_masked_labels_train = np.array(y_masked_labels_train)
    x_masked_test = np.array(x_masked_test)
    y_masked_labels_test = np.array(y_masked_labels_test)

    perm = np.random.permutation(len(x_masked_train))
    x_masked_train = x_masked_train[perm]
    y_masked_labels_train = y_masked_labels_train[perm]
    
    num_head = 2

    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
    sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
    encoder_output = word_embeddings + sinusoidal_embeddings

    for i in range(1):
        encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

    encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
    mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                            validation_split = 0.5, callbacks = [callback], 
                            epochs=2000, batch_size=5000, 
                            verbose=0)
    
    acc = []
    prob = []
    x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

    for sentence_number in x_test_subset:
        temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        temp = temp[:,-1,:]
        acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
        prob.append(temp[0][sentence_number[-1]])
        
    return (np.mean(acc), np.mean(prob))

In [9]:
accs_a = 0
probs_a = 0
accs_b = 0
probs_b = 0

for _ in range(10):
    
    (acc_a, prob_a) = get_accuracy_prob(10)
    
    print((acc_a, prob_a))
    
    accs_a += acc_a/10
    probs_a += prob_a/10
    
print((accs_a, probs_a))

(0.002, 0.011165041)
(0.014, 0.020805804)
(0.02, 0.022553742)
(0.007, 0.013524033)
(0.003, 0.009327709)
(0.012, 0.020110711)
(0.005, 0.017096343)
(0.008, 0.012117218)
(0.016, 0.017842272)
(0.002, 0.013445945)
(0.008900000000000002, 0.015798881836235523)


In [10]:
accs_a = 0
probs_a = 0
accs_b = 0
probs_b = 0

for _ in range(10):
    
    (acc_a, prob_a) = get_accuracy_prob(100)
    
    print((acc_a, prob_a))
    
    accs_a += acc_a/10
    probs_a += prob_a/10
    
print((accs_a, probs_a))

(0.0, 4.1165063e-07)
(0.0, 7.5572655e-07)
(0.0, 1.5380326e-06)
(0.0, 4.151977e-07)
(0.0, 1.5050199e-07)
(0.0, 3.6530378e-07)
(0.0, 1.3504804e-06)
(0.0, 1.3006141e-06)
(0.0, 2.7896747e-06)
(0.0, 2.013505e-06)
(0.0, 1.1090687536352562e-06)
